### Import Packages 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import xarray as xr
import seaborn as sns; sns.set(color_codes=True)
import pandas as pd
import netCDF4 as nc
from mpl_toolkits.basemap import Basemap
import csv

In [19]:
# Opening files with xarray

#Sea Ice Thickness 
sit_file = '/Users/fridaperez/Developer/repos/eeb-c177-project/sit.nc'  # prepping path for xr
sit_nc = nc.Dataset("envisat_SIT_fb_snow-AMSR_sh_2002_2011_ease2_w50000.nc") #reading SIT from nc 
data = xr.open_dataset(sit_file) # opening for xarray 


#Air Surface Temperature
tmp_file = '/Users/fridaperez/Developer/repos/eeb-c177-project/air.nc'
tmp_nc = nc.Dataset("air.mon.mean__M-O_2002-2011.nc")
data_air = xr.open_dataset(tmp_file)

#print(data)
#print(data_air)

In [3]:
#Opening files with NetCDF4

#reading in sit variables
time = sit_nc.variables['time'][:]
lat=sit_nc.variables['latitude'][:]
lon=sit_nc.variables['longitude'][:]
sit=sit_nc.variables['SIT'][:]

#reading in temp variables
lon_t = tmp_nc.variables["lon"][:]
lat_t = tmp_nc.variables["lat"][:]
level = tmp_nc.variables["level"][:]
air = tmp_nc.variables["air"][:]
tmp_time = tmp_nc.variables["time"]

In [22]:
#Using pandas to manipulate time for temperature dataset 
import datetime as dt
from netCDF4 import Dataset, date2index, num2date, date2num

dates_T=num2date(tmp_time[5:60],tmp_time.units)
dates_pd = pd.to_datetime(dates_T)
#print(dates_pd)

In [21]:
#Using pandas to manipulate time for SIT dataset 

time= data.time
time = pd.to_datetime(time.data)
#print(time)

 ## 1. For the SLR plot in 'R', we first subset the latitude and longitudinal area for Antarctica at the surface level 

In [28]:
# Slicing the 'z' axis, to only get the mean surface temp at 850 hPa where most convergence occurs
air_temp_at_surf=(air[:,2,:,:])
#print(air_temp_at_surf)
#Finding index value of certain longitudes
print(np.where(lat_t==-60))
print(np.where(lat_t==-90))
#air_temp_at_surf.shape
#Indexing to only get the Antarctic Circle
antcircle=air_temp_at_surf[5:60,60:72,:]
#print(antcircle)

(array([60]),)
(array([72]),)


## 2. Getting averages for surface temperature and sea ice thickness

In [8]:
#means for sit 
sit_mean=[]
for y in range(len(data.SIT)):
    mean = np.nanmean(data.SIT[y]) #avoids missing values through space 
    sit_mean.append(mean)

/Users/fridaperez/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


In [9]:
#means for temperature 
ant_mean=[]
for i in range(len(antcircle)):
    mean = np.mean(antcircle[i])
    ant_mean.append(mean)

## 3. R code: Using ggplot2

### Plot 1 

In [ ]:
#Reading in CSV for temperature and sit
sit_data<- read.csv("/Users/fridaperez/Desktop/sit_mean.csv")
tmp_data<-read.csv("/Users/fridaperez/Desktop/ant_mean_new.csv")

In [ ]:
# Creating a simple linear regression for SIT and temperature 
dd <- data.frame(sit_data=unlist(sit_data),tmp_data=unlist(tmp_data))
lmmodelsit<- lm(tmp_data~sit_data,dd) 
summary(lmmodelsit)

In [24]:
# Stating formula 
sit<-unlist(sit_data)
tmp<-unlist(tmp_data)
lm(formula = tmp~sit)

In [27]:
#Plotting and fitting the model 
p1 <- ggplot(lmmodelsit,aes(x=sit, y=tmp)) + geom_point(shape=1) + #assigning lm to "p1" geom_smooth(method=lm)
# Adding X and Y axis labels and a title to "p2", a new plot
p2 <- p1 + scale_x_continuous(name = "Sea Ice Thickness (in m)") +
    scale_y_continuous(name = "Surface Temperature (in F°)") + ggtitle("Surface Temperature Regression Line") 
#Adding the R-sqaured, alpha, and beta to the plot
p3<- p2+ annotate("text", x=1.45, y=-20.0, label = "R^2 == 0.69", parse=T) +
      annotate("text", x=1.45, y=-20.45, label = "alpha == 0.00", parse=T) +
      annotate("text", x=1.45, y=-20.90, label = "beta ==  -9.56", parse=T)
p3 #final plot

### Plot 2

In [ ]:
#To test if the SLR is a good fit and further evaluate the results of the linear regression we run the "autoplot" command in R, which allows us to analyze the residuals.
plt4<- autoplot(lmmodelsit,label.n = 0) 
plt4

### Plot 3

In [ ]:
#Reading in csv that contains SIT averages over time
sit_year<-read.csv("/Users/fridaperez/Desktop/sit_may_oct.csv")
sit_year<-as.data.frame(sit_year) #converting to data frame
print(sit_year)

In [30]:
#plotting with ggplot2
ggplot(sit_year) +geom_line(aes(x=sit_year$months,y=sit_year$X2003,color="2003"))+
geom_line(aes(x=sit_year$months,y=sit_year$X2004,color="2004"))+
geom_line(aes(x=sit_year$months,y=sit_year$X2005,color="2005"))+
geom_line(aes(x=sit_year$months,y=sit_year$X2006,color="2006"))+
geom_line(aes(x=sit_year$months,y=sit_year$X2007,color="2007"))+
geom_line(aes(x=sit_year$months,y=sit_year$X2008,color="2008"))+
geom_line(aes(x=sit_year$months,y=sit_year$X2009,color="2009"))+
geom_line(aes(x=sit_year$months, y=sit_year$X2010,color="2010"))+
geom_line(aes(x=sit_year$months,y=sit_year$X2011,color="2011"))+ 
ggtitle("Sea Ice Thickness") + scale_x_continuous(name = "May-October")+ 
scale_y_continuous(name = "Sea Ice Thickness (in m)")+
scale_colour_manual(name="Years",
            breaks = c("2003", "2004", "2005","2006", "2007","2008","2009", "2010", "2011"),
            values = c("2003"= "red", "2004"="darkolivegreen1", "2005"="blue", "2006"="yellow",
                       "2007"="purple","2008"="limegreen", "2009"="grey4","2010"="indianred1","2011"="steelblue")) 
            